Librerías, variables y funciones auxiliares para la ejecución de las celdas.

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Índices de los landmarks que definen el contorno de la cara
face_outline_indices = [
    10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 
    378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 
    162, 21, 54, 103, 67, 109
]

left_eye_indices = [
    33, 7, 163, 144, 145, 153, 154, 155, 133, 157, 158, 159, 160, 161, 246
]

right_eye_indices = [
    362, 398, 384, 385, 386, 387, 388, 466, 373, 374, 380
]

upper_mouth_indices = [
    61, 185, 40, 39, 37, 0, 267, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181
]

lower_mouth_indices = [
    146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 409, 270, 267, 0, 37, 39, 40, 185
]

# Función para redimensionar y rotar una imagen
def resize_and_rotate(image, width, height, angle_deg):
    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle_deg, 1.0)
    rotated = cv2.warpAffine(resized, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Función para ajustar dimensiones y recortar la imagen si es necesario
def adjust_and_crop(image, x, y, iw, ih):
    start_x = max(0, x)
    start_y = max(0, y)
    x_offset = 0 if x >= 0 else abs(x)
    y_offset = 0 if y >= 0 else abs(y)
    h_end = min(ih, y + image.shape[0]) - start_y
    w_end = min(iw, x + image.shape[1]) - start_x
    cropped = image[y_offset:y_offset + h_end, x_offset:x_offset + w_end]
    return start_x, start_y, cropped

# Función para superponer una imagen con transparencia
def overlay_transparent(base_image, overlay, x, y):
    h, w = overlay.shape[:2]
    for i in range(3):  # Para cada canal (BGR)
        base_image[y:y+h, x:x+w, i] = \
            base_image[y:y+h, x:x+w, i] * (1 - overlay[:, :, 3] / 255.0) + \
            overlay[:, :, i] * (overlay[:, :, 3] / 255.0)
        
# Función para crear un overlay basado en landmarks (ojos o boca)
def create_landmark_overlay(image, landmarks, indices_one, indices_two, dilation_size, color):
    ih, iw = image.shape[:2]
    mask = np.zeros((ih, iw), dtype=np.uint8)
    points_one = [(int(landmarks[i].x * iw), int(landmarks[i].y * ih)) for i in indices_one]
    points_two = [(int(landmarks[i].x * iw), int(landmarks[i].y * ih)) for i in indices_two]
    cv2.polylines(mask, [np.array(points_one)], isClosed=True, color=255, thickness=1)
    cv2.polylines(mask, [np.array(points_two)], isClosed=True, color=255, thickness=1)
    dilated_mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation_size, dilation_size)))
    overlay = np.zeros_like(image)
    overlay[dilated_mask == 255] = color
    return overlay



Demostración de los puntos clave que serán usados para posicionar el sombrero y el collar, así como el contorno de la cara

In [2]:
# Inicializar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Inicializar la cámara
cap = cv2.VideoCapture(0)

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB para procesarla con MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar landmarks faciales
        results = face_mesh.process(image_rgb)
        
        # Convertir la imagen de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Verificar si se detectaron caras
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Obtener los puntos específicos que forman el contorno de la cara
                ih, iw, _ = image_bgr.shape
                
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) 
                    for i in face_outline_indices
                ]

                cv2.polylines(image_bgr, [np.array(contour_points)], isClosed=True, color=(255, 255, 255), thickness=2)

                for i in face_outline_indices:
                    # Poner el número del landmark
                    if i == 54 or i == 284:
                        x = int(face_landmarks.landmark[i].x * iw)
                        y = int(face_landmarks.landmark[i].y * ih)
                        cv2.circle(image_bgr, (x, y), 5, (0, 0, 255), -1)

                # Dibujar una línea que une los landmarks 54 y 284
                ul_x, ul_y = int(face_landmarks.landmark[54].x * iw), int(face_landmarks.landmark[54].y * ih)    # Esquina superior izquierda (upper-left)
                ur_x, ur_y = int(face_landmarks.landmark[284].x * iw), int(face_landmarks.landmark[284].y * ih)  # Esquina superior derecha (upper-right)
                cv2.line(image_bgr, (ul_x, ul_y), (ur_x, ur_y), (0, 255, 0), 2)
                # Dibujar una X en el centro de la línea
                cx, cy = (ul_x + ur_x) // 2, (ul_y + ur_y) // 2
                cv2.line(image_bgr, (cx - 5, cy - 5), (cx + 5, cy + 5), (0, 255, 0), 2)
                cv2.line(image_bgr, (cx + 5, cy - 5), (cx - 5, cy + 5), (0, 255, 0), 2)


        # Mostrar el resultado
        cv2.imshow('Face Contour', image_bgr)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


Demostración de la detección del contorno de los ojos y la boca

In [3]:
# Inicializar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Inicializar la cámara
cap = cv2.VideoCapture(0)

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB para procesarla con MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar landmarks faciales
        results = face_mesh.process(image_rgb)
        
        # Convertir la imagen de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Verificar si se detectaron caras
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Obtener los puntos específicos que forman el contorno de la cara
                ih, iw, _ = image_bgr.shape
                
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) 
                    for i in face_outline_indices
                ]

                cv2.polylines(image_bgr, [np.array(contour_points)], isClosed=True, color=(255, 255, 255), thickness=2)

                eye_indices = left_eye_indices + right_eye_indices
                for i in eye_indices:
                    x = int(face_landmarks.landmark[i].x * iw)
                    y = int(face_landmarks.landmark[i].y * ih)
                    cv2.circle(image_bgr, (x, y), 2, (0, 0, 255), -1)
                
                mouth_indices = upper_mouth_indices + lower_mouth_indices
                for i in mouth_indices:
                    x = int(face_landmarks.landmark[i].x * iw)
                    y = int(face_landmarks.landmark[i].y * ih)
                    cv2.circle(image_bgr, (x, y), 2, (0, 0, 255), -1)


        # Mostrar el resultado
        cv2.imshow('Face Contour', image_bgr)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

Aplicación de filtros para los landmarks y superposición de imágenes respecto a los puntos de interés del borde superior de la cara

In [6]:
# Inicializar MediaPipe Face Mesh (para obtener los landmarks)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y collar (sin cambios)
hat = cv2.imread('./images/hat_crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape

ruff = cv2.imread('./images/ruff_crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = ruff.shape

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Inicializar Face Mesh
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_faces=2) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para obtener los landmarks de la cara
        results = face_mesh.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:

                # --- CÁLCULO DE MEDIDAS PARA LA POSICIÓN Y ROTACIÓN DE LOS OBJETOS ---
                # Dimensiones de la imagen
                ih, iw, _ = image_bgr.shape
                
                # Obtener los puntos del contorno de la cara
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in face_outline_indices
                ]

                # Calcular tamaño y posición del rectángulo que contiene la cara a partir del contorno
                x, y, w, h = cv2.boundingRect(np.array(contour_points))
                

                # Obtener rotación de la cara a partir de la línea superior de la cara, trazada entre los landmarks 54 y 284
                ul_x, ul_y = int(face_landmarks.landmark[54].x * iw), int(face_landmarks.landmark[54].y * ih)    # Esquina superior izquierda (upper-left)
                ur_x, ur_y = int(face_landmarks.landmark[284].x * iw), int(face_landmarks.landmark[284].y * ih)  # Esquina superior derecha (upper-right)

                # Calcular la diferencia en x e y para obtener el ángulo de rotación de la cara
                dx = ur_x - ul_x
                dy = ur_y - ul_y

                # Calcular el ángulo de rotación, como el ángulo entre la línea y el eje horizontal
                angle_deg = -np.degrees(np.arctan2(dy, dx))             # Negativo debido a la inversión de en la imagen de la cámara
                
                
                # --- POSICIONAMIENTO DEL SOMBRERO Y COLLAR EN LA CARA ---
                # Tomamos el punto medio de la línea superior de la cara como referencia
                center_x = (ul_x + ur_x) // 2
                center_y = (ul_y + ur_y) // 2

                # Hallamos la longitud de la línea superior de la cara para usarla de referencia para redimensionar objetos
                hypotenuse = math.sqrt(dy ** 2 + dx ** 2)


                # --- SOMBRERO ---
                # Redimensionado
                new_hat_w = int(hypotenuse)*2                       # Sombrero 2 veces más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))        # Mantener proporción para el alto

                # Rotación del sombrero acorde al ángulo de la cara
                rotated_hat = resize_and_rotate(hat, new_hat_w, new_hat_h, angle_deg)
                
                rotated_hat_h, rotated_hat_w, _ = rotated_hat.shape

                # Posición inicial del sombrero en la imagen, centrado con el punto medio de los ojos
                x_hat = center_x - (rotated_hat_w // 2)
                y_hat = center_y - (rotated_hat_h // 2) - int(0.3 * h)  # Posicionar el sombrero justo encima de la cara con este pequeño desplazamiento

                # Ajustar dimensiones y recortar si se sale de los límites de la imagen
                x_hat, y_hat, rotated_hat = adjust_and_crop(rotated_hat, x_hat, y_hat, iw, ih)

                # Superponer el sombrero con transparencia
                overlay_transparent(image_bgr, rotated_hat, x_hat, y_hat)


                # --- COLLAR ---
                # Redimensionado
                new_ruff_w = int(w * 1.8)                            # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))     # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado respecto a la cara
                x_ruff = x - int((new_ruff_w - w) / 2)              # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.15 * new_ruff_h)             # Posicionar el collar justo debajo de la cara con este pequeño desplazamiento

                # Ajustar dimensiones y recortar si se sale de los límites de la imagen
                x_ruff, y_ruff, resized_ruff = adjust_and_crop(resized_ruff, x_ruff, y_ruff, iw, ih)

                # Superponer el collar con transparencia
                overlay_transparent(image_bgr, resized_ruff, x_ruff, y_ruff)
                
                
                # --- DIBUJAR CONTORNOS DE OJOS Y BOCA ---
                # Dibujar contorno de los ojos y de la boca
                eye_overlay = create_landmark_overlay(image_bgr, face_landmarks.landmark, left_eye_indices, right_eye_indices, 5, (0, 80, 255))
                mouth_overlay = create_landmark_overlay(image_bgr, face_landmarks.landmark, upper_mouth_indices, lower_mouth_indices, 10, (0, 0, 255))
                combined_overlay = cv2.addWeighted(eye_overlay, 0.5, mouth_overlay, 0.5, 0)
                image_bgr = cv2.addWeighted(combined_overlay, 0.4, image_bgr, 1, 0)


        # Mostrar la imagen con el filtro aplicado
        cv2.imshow('Face Filter', image_bgr)

        # Salir si se presiona la tecla 'q'
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
